In [1]:
import requests
from bs4 import BeautifulSoup
import json
import openpyxl

In [2]:
def guardar_excel_edicion(titulo, año, ganadores_categoria, artistas):
    """
    Función para guardar un archivo Excel con la información de los ganadores por 
    cada categoría y todos los artistas nominados de una edición de los Premios Grammy
    """
    # Crea un nuevo archivo Excel
    wb = openpyxl.Workbook()
    hoja_activa = wb.active
    hoja_activa.title = "Categorias y Ganadores"

    # Itera sobre la lista de ganadores y escribe los datos en columnas
    for indice, dato in enumerate(ganadores_categoria.items(), start=1):
        categoria, ganador = dato
        hoja_activa.cell(row=indice, column=1, value=categoria) # Escribe la categoría en la primera columna
        if type(ganador) is list: # Si hay varios ganadores en una categoría
            for g in range(len(ganador)):
                hoja_activa.cell(row=indice, column=g+2, value=ganador[g]) # Escribe cada ganador en las columnas posteriores
        else: # Si solo hay un ganador en una categoría
            hoja_activa.cell(row=indice, column=2, value=ganador)

    # Añade la segunda hoja con los artistas nominados al archivo Excel
    nueva_hoja = wb.create_sheet("Artistas nominados")

    # Itera sobre la lista de artistas nominados y escribe los datos en la primera columna
    for indice, nominado in enumerate(artistas, start=1):
        nueva_hoja.cell(row=indice, column=1, value=nominado)

    # Guarda el libro de Excel en función del año que estemos estudiando
    nombre = str(titulo) + ' ' + str(año) + ".xlsx"
    wb.save(nombre)

In [3]:
def grammy_URL(n): 
    """
    Función para obtener una lista "ganadores_categoria", donde para cada categoría de la edición
    se indica qué artistas ganaron y una lista "artistas", donde se incluyen todos los artistas 
    nominados a mínimo un premio Grammy.
    El parámetro n indica las últimas n ediciones de los premios Grammy a descargar.
    """
    url_grammy = 'https://www.grammy.com/'
    ult_edicion = 'awards/66th-annual-grammy-awards-2023'

    # Hace la solicitud inicial para obtener la lista de todas las ediciones
    req = requests.get(str(url_grammy)+str(ult_edicion))
    soup = BeautifulSoup(req.content, 'lxml')
    data = soup.find_all('script')[-1]  # La informacion estan en el último script, todo lo demas es formato de la pagina
    d = json.loads(data.contents[0])  # Convierte el string JSON a un diccionario
    ediciones = d['props']['pageProps']['pageContent']['getAwardsYearsList']['hits'] # URLS DE TODAS LAS EDICIONES DE LOS GRAMMYS

    # Itera sobre los primeros n años
    for i in range(n):
        edicion = ediciones[i]
        titulo = edicion['title']
        id = edicion['slug']
        url = url_grammy + id

        # Solicitud para obtener la informacion de ese año
        req_edicion = requests.get(url)
        soup_edicion = BeautifulSoup(req_edicion.content, 'lxml')
        data_edicion = soup_edicion.find_all('script')[-1]
        d_edicion = json.loads(data_edicion.contents[0])
        hits = d_edicion['props']['pageProps']['pageContent']['getAwardsYears']['hits'][0]
        año = d_edicion['props']['pageProps']['pageContent']['getAwardsYears']['hits'][0]['showYear']

        # Inicializamos las variables de artistas y ganadores por categoria
        artistas = set()
        ganadores_categoria = {}

        # Para cada categoria de la edicion
        len_categorias = len(hits['categoryDetails'])
        for i_categoria in range(len_categorias):
            categoria = hits['categoryDetails'][i_categoria]
            nombre_categoria = categoria['title'][0]['name']  # Nombre de la categoria
            len_nominados = len(categoria['nominations'])  # Numero de nominados por categoria
            
            # Para cada nominado en la categoria
            for i_nominado in range(len_nominados):
                nominado = categoria['nominations'][i_nominado]  # Nº de personas que se referencian en una
                participantes = len(nominado['creditedArtists']) # nominacion (cantantes, escritores, productores, etc.)
                
                # Para cada participante en la nominacion
                for i_participante in range(participantes):
                    participante = nominado['creditedArtists'][i_participante] # Referencia a la persona
                    
                    if participante['creditedAs'] == 'Artist':  # Si la persona es un artista, lo añadimos al set de artistas
                        artista = participante['title']
                        artistas.add(artista)
                    
                    if nominado['isWinner']:  # Si es un ganador, lo añadimos junto con la categoria que ha ganado
                        ganador = participante['title']
                        if nombre_categoria in ganadores_categoria.keys(): # Si la categoria ya existe en el diccionario, comprobamos que los 
                            if participante['creditedAs'] == 'Artist':     # demás participantes sean artistas, para no añadir productores, etc.
                                ganadores_categoria[nombre_categoria].append(ganador) # Esto es porque algunas categorías no son premios a artistas.
                        
                        if nombre_categoria not in ganadores_categoria.keys(): # Si la categoria no existe en el diccionario, 
                            ganadores_categoria[nombre_categoria] = []         # la creamos y añadimos el ganador
                            ganadores_categoria[nombre_categoria].append(ganador)

        # Guardamos la información en un archivo Excel
        guardar_excel_edicion(titulo, año, ganadores_categoria, artistas)

        # Imprime la información de la edición
        print(f'Edición {titulo}')
        print(f'- Nº artistas: {len(artistas)}')
        print(f'- Nº categorías: {len(ganadores_categoria)}')
        print(f"- Archivo generado: {str(titulo) + ' ' + str(año) + '.xlsx'}")
        print('-' * 50)

In [4]:
# Llamar a la función para los últimos 5 años
grammy_URL(5)

Edición 66th Annual GRAMMY Awards
- Nº artistas: 303
- Nº categorías: 94
- Archivo generado: 66th Annual GRAMMY Awards 2023.xlsx
--------------------------------------------------
Edición 65th Annual GRAMMY Awards
- Nº artistas: 304
- Nº categorías: 91
- Archivo generado: 65th Annual GRAMMY Awards 2022.xlsx
--------------------------------------------------
Edición 64th Annual GRAMMY Awards
- Nº artistas: 281
- Nº categorías: 86
- Archivo generado: 64th Annual GRAMMY Awards 2021.xlsx
--------------------------------------------------
Edición 63rd Annual GRAMMY Awards
- Nº artistas: 264
- Nº categorías: 84
- Archivo generado: 63rd Annual GRAMMY Awards 2020.xlsx
--------------------------------------------------
Edición 62nd Annual GRAMMY Awards
- Nº artistas: 260
- Nº categorías: 84
- Archivo generado: 62nd Annual GRAMMY Awards 2019.xlsx
--------------------------------------------------
